# Google Drive

You can access files in Drive using the [native REST API](https://developers.google.com/drive/v3/web/about-sdk) or a wrapper like [PyDrive](https://googledrive.github.io/PyDrive/docs/build/html/index.html).

## Drive REST API

The first step is to authenticate.

In [0]:
from google.colab import auth
auth.authenticate_user()

Now we can construct a Drive API client.

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

With the client created, we can use any of the functions in the [Google Drive API reference](https://developers.google.com/drive/v3/reference/). Examples follow.


## Downloading data from a Drive file into Python

In [0]:
# Download the file we just uploaded.
#
# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1uBtlaggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'target_file_id'

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: my sample file


In [0]:
# second way of doing this
file_id = 'target_file_id'
#-> dunno what that is'1ZdR3L3qP4Bkq8noWLJHSr_iBau0DNT4Kli4SxNc2YEo'
request = drive_service.files().export_media(fileId=file_id,
                                             mimeType='application/pdf')
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print "Download %d%%." % int(status.progress() * 100)

### Look at me!
Derek waz here

In [0]:
# gets text file (.txt)
# actually gets the file in memory
def get_drive_file(file_id):
  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
    _, done = downloader.next_chunk()

  downloaded.seek(0)
  #print('Downloaded file contents are: {}'.format(downloaded.read()))

  txtFile = io.TextIOWrapper(downloaded)
  return txtFile

In [0]:
####
# WIP
# Effiency Unknown
# Depdencies: opencv (cv2)
# actually gets the file in memory
#####
from io import BytesIO
from googleapiclient.http import MediaIoBaseDownload
from cv2 import imdecode

def download_image(file_id):
  request = drive_service.files().get_media(fileId=file_id)
  downloaded = BytesIO()#io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = downloader.next_chunk()

  downloaded.seek(0)

  # improvements possible???
  # there is likely two copies used and three copies in memory
  # first convert into numpy byte array
  foo = np.frombuffer(downloaded.getvalue(), dtype=np.uint8)
  # -1 means unchanged I think
  drive_image = imdecode(foo, -1)
  return drive_image

In [0]:
# effiency unkown
# maps filename to a list of files
# USING HTTP
# I think returns array of {name, id}
# looks in the color folder if true
# does not actually get contents in memory
def get_files(name, color):
  # CHANGE THIS LINE TO CHANGE (PARENT) FOLDER
  parent = 'RGB_FOLDER_ID' if color else 'MASK_FOLDER_ID'
  query = "'{0}' in parents".format(parent)
  # note there are mutually inclusive variables for team drive
  # if you take out any you may get 403
  response = drive_service.files().list(#q="mimeType='image/jpeg'",
                                        q=query,
                                        spaces='drive',
                                        fields='nextPageToken, files(id, name)',
                                        pageToken=None,#page_token,
                                        supportsTeamDrives=True,
                                        corpora='teamDrive',
                                        includeTeamDriveItems=True,
                                        teamDriveId='0AI79WEVW3y_yUk9PVA'
  ).execute()
  return response.get('files', [])
#########
#READ THIS
#########
# id = get_files("171206_034513181_Camera_6.jpg", True)[0].get('id')
get_files("171206_034513181_Camera_6.jpg", True).next_link
# img = download_image(id)